In [ ]:
from pathlib import Path
import geopandas
import pandas as pd

In [ ]:
DATA_PATH = Path("data/")
CRS = "EPSG:4326"

FILENAMES = {
    "RAR": "lcar000b21a_e.shp", 
    "CD": "ldr_000b21a_f.shp"
}

In [ ]:
level = "CD"

In [ ]:
df_farms = pd.read_csv(
    DATA_PATH / "fermes" / f"{level}_2021.csv", delimiter=";"
).drop(
    [
        "UNITÉ DE MESURE", 
        "IDENTIFICATEUR D'UNITÉ DE MESURE", 
        "FACTEUR SCALAIRE", 
        "VECTEUR", 
        "COORDONNÉES", 
        "IDENTIFICATEUR SCALAIRE", 
        "STATUS", 
        "SYMBOLE", 
        "TERMINÉ", 
        "DÉCIMALES"
    ], 
    axis=1
).reset_index()

df_farms["GÉO"] = df_farms.apply(
    lambda row: row["GÉO"].split(" [")[0], 
    axis=1
)

df_farms["DGUID"] = df_farms.apply(
    lambda row: row["DGUID"][4:], 
    axis=1
)


df_farms = df_farms.replace(
    "Nombre d'exploitations déclarantes", 
    "num_exploitations"
).replace(
    "Nombre d'entailles", 
    "num_entailles"
).replace(
    "Nombre de fermes déclarantes", 
    "num_exploitations"
).pivot(
    index=["GÉO", "DGUID"], 
    columns=["Entailles d'érables"], 
    values="VALEUR"
)

df_farms.head()

In [ ]:
df_boundaries = geopandas.read_file(
    DATA_PATH / "boundaries" / FILENAMES[level]
).to_crs(
    CRS
).drop(
    ["DRIDU", "DRGENRE", "SUPTERRE"], 
    axis=1
).rename(
    columns={
        "DRNOM": "area_name", 
        "IDUGD": "DGUID", 
        "PRIDU": "PRUID"
    }
)

df_boundaries["DGUID"] = df_boundaries.apply(
    lambda row: row["DGUID"][4:], 
    axis=1
)

df_boundaries.head()

In [ ]:
df = pd.merge(
    df_boundaries, 
    df_farms, 
    on="DGUID"
)

df.head()

In [ ]:
df.to_file(f'entailles_{level}.shp') 